# Multi-Scenario MORDM

Multi-scenario MORMD is an extension of normal MORDM to better include robustness considerations within the search phase. It starts from the scenario discovery results resulting from MORDM. Next, from the experiments within this box, a set of scenarios is selected. 

There are many ways of selecting the additional scenarios. The original paper which introduced multi-scenario MORMD [Watson and Kaspzryk (2017)](https://doi.org/10.1016/j.envsoft.2016.12.001) did it in a more or less adhoc manner. [Eker and Kwakkel (2018)](https://doi.org/10.1016/j.envsoft.2018.03.029) introduced a more formal selection approach, the code of which can be found on [GitHub](https://github.com/sibeleker/MORDM---Multi-scenario-search). 

For this assignment, make an informed selection of 4 scenarios, using an approach of your choice. Motivate carefully your selection procedure. 


In [ ]:
from ema_workbench import load_results

In [1]:
results, outcomes = load_results('./results/selected_results.tar.gz')

NameError: name 'load_results' is not defined

In [ ]:
ema_logging.log_to_stderr(ema_logging.INFO)

lake_model = Model('lakemodel', function=lake_problem)
lake_model.time_horizon = 100

lake_model.uncertainties = [RealParameter("mean", 0.01, 0.05),
                            RealParameter("stdev", 0.001, 0.005),
                            RealParameter("b", 0.1, 0.45),
                            RealParameter("q", 2, 4.5),
                            RealParameter("delta", 0.93, 0.99)]

lake_model.levers = [RealParameter("c1", -2, 2),
                     RealParameter("c2", -2, 2),
                     RealParameter("r1", 0, 2),
                     RealParameter("r2", 0, 2),
                     RealParameter("w1", 0, 1)]

lake_model.outcomes = [ScalarOutcome("max_P", kind=ScalarOutcome.MINIMIZE, expected_range=(0, 5)),
                       ScalarOutcome("utility", kind=ScalarOutcome.MAXIMIZE, expected_range=(0, 2)),
                       ScalarOutcome("inertia", kind=ScalarOutcome.MAXIMIZE, expected_range=(0, 1)),
                       ScalarOutcome("reliability", kind=ScalarOutcome.MAXIMIZE, expected_range=(0, 1))]

lake_model.constants = [Constant("alpha", 0.4),
                        Constant("nsamples", 100),
                        Constant("myears", 100)]

n_scenarios = 10

### Scenario Selection
The way in which the scenario will be selected will be by arbitrarily choosing the scenario with the highest utility in the set, the lowest and two closest to the median value of utility

In [ ]:
df_outcomes = pd.DataFrame(outcomes)
median = df_outcomes['utility'].median()

df_median = df_outcomes.iloc[((df_outcomes['utility']-median).abs().argsort()[:2])]
df_outcomes = df_outcomes.loc[(df_outcomes['utility'] == max(df_outcomes['utility'])) |
                              (df_outcomes['utility'] == min(df_outcomes['utility']))]
df_outcomes = pd.concat([df_outcomes, df_median])
index_list = df_outcomes.index.tolist()
results = results.iloc[index_list].reset_index(drop=True)
df_outcomes.reset_index(drop=True, inplace=True)

## Search for each scenario

For each of the four selected scenarios, use many-objective optimization to find a pareto approximate set using the same approach as for assignment 8. Remember to check for convergence (and time permitting, seed analysis), and be careful in what epsilon values to use (not to coarse, not too small). 

Store the resulting set of pareto solutions in a smart way for subsequent analysis.


In [ ]:
convergence_metrics = [HyperVolume.from_outcomes(lake_model.outcomes),
                           EpsilonProgress()]

with MultiprocessingEvaluator(lake_model) as evaluator:
    results_, convergence = evaluator.optimize(nfe=10000, searchover='levers',
                                               epsilons=[0.1, 0.1, 0.05, 0.05],
                                               convergence=convergence_metrics)

fig, (ax1, ax2) = plt.subplots(ncols=2, sharex=True, figsize=(8,4))
ax1.plot(convergence.nfe, convergence.epsilon_progress)
ax1.set_ylabel('$\epsilon$-progress')
ax2.plot(convergence.nfe, convergence.hypervolume)
ax2.set_ylabel('hypervolume')

ax1.set_xlabel('number of function evaluations')
ax2.set_xlabel('number of function evaluations')
plt.show()

In [ ]:
outcomes = results.loc[:, ['max_P', 'utility', "inertia", 'reliability']]
limits = parcoords.get_limits(outcomes)
axes = parcoords.ParallelAxes(limits)
axes.plot(outcomes)

# we invert this axis so direction of desirability is the same
axes.invert_axis('max_P')
plt.show()

From the plots generated by these models, it is clear that hypervolume stabilized around 4,000 function evaluations. However, the epsilon progress has stabilized just before 10,000 and the nfe will therefore be kept at 10,000, as any increase in this will simply be a waste of resources and time.

## Re-evaluate under deep uncertainty

Combine the pareto set of solutions found for each scenario. Next, turn each solution into a policy object. If you have a very large number of policies, you can choose to down sample your policies in some reasoned way (*e.g.*, picking min and max on each objective, slicing across the pareto front with a particular step size). As a rule of thumb, try to limit the set of policies to at most 50. 

Re-evaluate the combined set of solutions over 1000 scenarios sampled using LHS.


In [33]:
policy_results = new_results.drop(['max_P', 'utility', "inertia", 'reliability'], axis=1)
new_results_dict = new_results.to_dict('index')
policies = []
for dict_index in range(len(new_results_dict)):
    policy_dict = new_results_dict[dict_index]
    policies.append(Policy(f"pol_{dict_index}", **policy_dict))

Having run the model multiple times, the number of policies has never exceeded 50 and therefore, all the policies will be used.

In [ ]:
n_scenarios = 1000
LHS = 'lhs'
with MultiprocessingEvaluator(lake_model) as evaluator:
    results__, outcomes__ = evaluator.perform_experiments(scenarios=n_scenarios,
                                                          policies=policies,
                                                          uncertainty_sampling=LHS)

Calculate both the maximum regret, and the domain criterion using the values provided in [Bartholomew and Kwakkel (2020)](https://doi.org/10.1016/j.envsoft.2020.104699). Ignore the max_P objective.

visualize the results in parallel coordinate plot. 

Are there any promising compromise solutions which balance performance in both the reference scenarios as well as in terms of their robustness?


In [ ]:
def calculate_regret(outcomes_w_policy, results_w_policy) :
    df_outcomes_w_policy = pd.DataFrame(outcomes_w_policy)

    results_w_policy.rename(columns={"max_P": "max_P_initial",
                                     "inertia": "inertia_initial",
                                     "utility": "utility_initial",
                                     "reliability": "reliability_initial"},
                            inplace=True)
    df_full = pd.concat([df_outcomes_w_policy, results_w_policy], axis=1)
    df_regret = pd.DataFrame(columns=['policy_num', 'max_P', 'utility', "inertia", 'reliability'])
    for unique_item in df_full['policy'].unique() :
        df_partial = df_full.loc[df_full['policy'] == unique_item]
        df_partial.reset_index(drop=True, inplace=True)
        max_max_P = max(df_partial['max_P'])
        max_utility = max(df_partial['utility'])
        max_inertia = max(df_partial['inertia'])
        max_reliability = max(df_partial['reliability'])
        for index in range(len(df_partial)) :
            loc_max_p = df_partial.loc[index, 'max_P']
            loc_utility = df_partial.loc[index, 'utility']
            loc_inertia = df_partial.loc[index, 'inertia']
            loc_reliability = df_partial.loc[index, 'reliability']
            new_row = {'policy_num' : unique_item,
                       'max_P' : max_max_P - loc_max_p,
                       'utility' : max_utility - loc_utility,
                       'inertia' : max_inertia - loc_inertia,
                       'reliability' : max_reliability - loc_reliability}
            df_regret = df_regret.append(new_row, ignore_index=True)

    columns_to_sum = ['max_P', 'utility', "inertia", 'reliability']
    df_regret['total_regret'] = df_regret[columns_to_sum].sum(axis=1)

    return df_regret
df_regret = calculate_regret(outcomes__, results__)

df_regret_plot = pd.DataFrame(columns=['policy', 'max_regret', 'min_regret'])
for unique_item in df_regret['policy_num'].unique() :
    df_partial = df_regret.loc[df_regret['policy_num'] == unique_item]
    new_row = {'policy' : unique_item,
               'max_regret' : max(df_partial['total_regret']),
               'min_regret' : min(df_partial['total_regret'])}
    df_regret_plot = df_regret_plot.append(new_row, ignore_index=True)

In [ ]:
df_regret_plot.plot(y='max_regret', style='o', use_index=True)
plt.show()

From the runs conducted within the program, the best solutions that have been found have c1 and c2 values that are positive around 0.5-0.7, r1 and r2 which are highly positive 1.5-1.7 and a very low w1 at around 0.05-0.06